In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

df = pd.read_csv("/content/sample_data/user_personalized_features.csv")

In [ ]:
# Drop index column if present
df = df.drop(columns=["Unnamed: 0"], errors="ignore")

In [ ]:
# Strip whitespace for string cols
for c in df.select_dtypes(include="object").columns:
    df[c] = df[c].astype(str).str.strip().replace({"nan": None})

In [ ]:
# Normalize newsletter column to boolean
df["Newsletter_Subscription"] = (
    df["Newsletter_Subscription"]
    .astype(str).str.lower()
    .map({"true": True, "false": False, "1": True, "0": False, "yes": True, "no": False})
).fillna(False).astype(bool)

In [ ]:
# Convert numeric columns and fill missing with median
numeric_cols = ["Age","Income","Last_Login_Days_Ago","Purchase_Frequency",
                "Average_Order_Value","Total_Spending","Time_Spent_on_Site_Minutes","Pages_Viewed"]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        df[c] = df[c].fillna(df[c].median())

In [ ]:
# Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

In [ ]:
# Clip outliers (1st - 99th percentile)
for c in numeric_cols:
    if c in df.columns:
        lo = df[c].quantile(0.01); hi = df[c].quantile(0.99)
        df[c] = df[c].clip(lo, hi)

In [ ]:
# Engagement and implicit rating
df["engagement"] = df.get("Time_Spent_on_Site_Minutes",0) + df.get("Pages_Viewed",0)
scaler = MinMaxScaler()
scale_cols = [c for c in ["Purchase_Frequency","Total_Spending","engagement"] if c in df.columns]
df[[col+"_norm" for col in scale_cols]] = scaler.fit_transform(df[scale_cols])
df["implicit_rating"] = (df["Purchase_Frequency_norm"]*0.45 +
                         df["Total_Spending_norm"]*0.45 +
                         df["engagement_norm"]*0.10)

In [ ]:
# Bucketize
df["AgeBucket"] = pd.cut(df["Age"], bins=[0,18,25,35,45,55,65,120],
                         labels=["<18","18-24","25-34","35-44","45-54","55-64","65+"])
try:
    df["IncomeBucket"] = pd.qcut(df["Income"], q=3, labels=["Low","Medium","High"])
except:
    df["IncomeBucket"] = pd.cut(df["Income"], bins=3, labels=["Low","Medium","High"])

In [ ]:
# Label encode category for modeling
le = LabelEncoder()
df["category_id"] = le.fit_transform(df["Product_Category_Preference"].astype(str))

In [ ]:
# Save cleaned CSV
df.to_csv("clean_dataset.csv", index=False)

In [ ]:
from google.colab import files
files.download("clean_dataset.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>